In [2]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
import folium
import numpy as np
from tqdm.notebook import tqdm
from utils.graph_creator import create_road_graph, add_nodes_to_graph
from utils.population_count import count_population_for_houses
from utils.flow_graph_creator import create_flow_graph
import pickle
import momepy
from momepy.datasets import get_path
import geopandas as gpd
import networkx as nx
from geopy.distance import geodesic
from shapely.geometry import Point
from rtree import index
import tqdm.notebook as tqd

In [3]:
with open("graph.pkl", "rb") as f:
    Graph = pickle.load(f)

In [4]:
df = count_population_for_houses(path='/Users/user/PycharmProjects/urban-graph-forecasting/data/Дома_исходные.shp', G=Graph)

/Users/user/numericalMethods/.venv/urban-graph-forecasting/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: /Users/user/PycharmProjects/urban-graph-forecasting/data/Дома_исходные.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [9]:
df

,Name,Type,Caption,Purpose,PostIndex,Elevation,Entrances,Apartments,District,DistrictId,...,Street3,StreetId3,Number3,Street4,StreetId4,Number4,Material,geometry,Nearest_Node,Population
0,None,Жилые дома,None,Таунхаус,108803,2.0,NaN,NaN,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Железобетон,"POLYGON ((37.41758 55.5224, 37.4174 55.52243, ...","(37.420997170530605, 55.52348633341077)",5
1,None,Жилые дома,None,Жилой дом,108803,10.0,3.0,108.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Панель,"POLYGON ((37.44152 55.53147, 37.44171 55.53075...","(37.44139018512153, 55.530647048820775)",324.0
2,None,Жилые дома,None,Жилой дом,108803,9.0,2.0,67.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Кирпич,"MULTIPOLYGON (((37.44603 55.53096, 37.44615 55...","(37.44629911882313, 55.530645828624)",201.0
3,None,Жилые дома,None,Малоэтажный жилой дом,108803,3.0,3.0,18.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Кирпич,"POLYGON ((37.4441 55.52626, 37.44313 55.52617,...","(37.44368466202023, 55.52638863671055)",54.0
4,None,Жилые дома,None,Малоэтажный жилой дом,108803,3.0,2.0,24.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Кирпич,"POLYGON ((37.44684 55.52567, 37.4463 55.52562,...","(37.44663796334831, 55.52586846809827)",72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11745,None,Частные дома,None,Частный дом,142704,1.0,NaN,NaN,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Дерево,"POLYGON ((37.47576 55.60417, 37.47564 55.60423...","(37.475084643955924, 55.60424756216844)",5
11746,None,Частные дома,None,Частный дом,142704,1.0,NaN,NaN,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Дерево,"POLYGON ((37.47439 55.60366, 37.47448 55.60362...","(37.47400936056083, 55.60350320757983)",5
11747,None,Частные дома,None,Частный дом,108802,2.0,NaN,NaN,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Кирпич,"POLYGON ((37.4659 55.58782, 37.46606 55.58781,...","(37.46572509701068, 55.587651669318966)",5
11748,None,Частные дома,None,Частный дом,108802,2.0,NaN,NaN,Коммунарка,4504209520926899,...,None,None,None,None,None,None,Кирпич,"POLYGON ((37.46996 55.58982, 37.4701 55.58986,...","(37.46937468251547, 55.58979761728818)",5


In [ ]:
def balance_dict(values):
    """
    Корректирует значения словаря так, чтобы их сумма была равна нулю,
    добавляя равномерно распределенную разницу к каждому значению.

    Args:
        values (dict): Словарь с числовыми значениями.

    Returns:
        dict: Скорректированный словарь с суммой значений, равной нулю.
    """
    total_sum = sum(values.values())

    # Проверка, требуется ли корректировка
    if total_sum != 0:
        adjustment = -total_sum / len(values)
        values = {k: v + adjustment for k, v in values.items()}

    return values


def shortest_path_to_type(graph,
                          start,
                          target_type
                          ):
    # Проходим по всем путям от начальной вершины
    for node in nx.bfs_tree(graph, start):
        # Если достигли вершины с нужным типом, возвращаем путь
        if graph.nodes[node].get('type') == target_type:
            return nx.shortest_path(graph, start, node)[-1]
    return None  # Если такой вершины нет


def create_flow_graph(G,
                      house_dataframe,
                      target_types=('school', 'stop', 'metro'),
                      population_types_summer=None,
                      population_types_winter=None,
                      season=True
                      ):
    """
    Create a flow graph from a house df with counted population and nearest roads
    :param population_types_winter:
    :param population_types_summer:
    :param G:
    :param house_dataframe: датафрейм с ближайшими дорогами для домов
    :param target_types: категории для стоков
    :param season: пересчет в зависимости от погоды
    :return: demands
    """

    if population_types_summer is None:
        population_types_summer = {'school': 0.1,
                                   'stop': 0.25,
                                   'metro': 0.25}
    if population_types_winter is None:
        population_types_winter = {'school': 0.1,
                                   'stop': 0.3,
                                   'metro': 0.3}

    sources = set(house_dataframe['Nearest_Node'].to_list())  # Источники
    sinks = set([node for node, data in G.nodes(data=True) if data['type'] in target_types])  # Стоки
    demands_keys = sources | sinks
    demands = dict.fromkeys(demands_keys, 0)
    for key in tqdm(demands_keys):
        near = house_dataframe[house_dataframe['Nearest_Node'] == key]
        near_list = near['Population'].to_list()
        if season:
            demands[key] -= int(sum(near_list) * 0.6)
        else:
            demands[key] -= int(sum(near_list) * 0.5)
        if season:
            population = sum(near['Population'].to_list())
            target_nodes = dict.fromkeys(demands_keys, None)
            for node_type in target_types:
                target_nodes[node_type] = shortest_path_to_type(G, key, node_type)
            for keys in target_types:
                try:
                    demands[target_nodes[keys]] += int(population_types_summer[keys] * population)
                except:
                    pass
        else:
            population = near['Population'].to_list()
            target_nodes = dict.fromkeys(demands_keys, None)
            for node_type in target_types:
                target_nodes[node_type] = shortest_path_to_type(G, key, node_type)
            for keys in target_types:
                demands[target_nodes[keys]] += int(population_types_winter[keys] * population)

    demands = balance_dict(demands)
    return demands

In [ ]:
create_flow_graph(Graph, df)

  6%|▌         | 162/2897 [01:14<22:17,  2.05it/s]